# Exploratory analysis of dataset

In [ ]:
from typing import Any, List
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import sys
import os

### Define data path

In [ ]:
data_path = "/Users/manuel/Desktop/BiomedDataAnalysisCourse/project/data/"
ps_data_path = os.path.join(data_path, "database_base.csv")
rx_data_path = os.path.join(data_path, "database_RX_torace.csv")

### Define functions used throughout the analysis

In [ ]:
def compute_nan_pct(data: List) -> float:
    assert isinstance(data, list)
    assert bool(data)
    nan_num = [1 for d in data if str(d) == "nan"]
    nan_pct = sum(nan_num) / len(data)
    return nan_pct

In [ ]:
def nan_to_zero(value: str) -> int:
    if str(value) == "nan":
        return 0
    value = int(value)
    assert value == 1
    return value

In [ ]:
def compute_date_diff(start: str, stop: str) -> int:
    assert isinstance(start, str)
    assert isinstance(stop, str)
    start = datetime.strptime(start, "%d/%m/%Y")
    stop = datetime.strptime(stop, "%d/%m/%Y")
    if start > stop:
        print(start, stop)
    diff = stop - start
    return diff.days

In [ ]:
def compute_death_response(death_date: str) -> int:
    assert isinstance(death_date, str)
    if str(death_date) == "nan":
        return 0
    return 1

In [ ]:
def make_barplot(df: pd.DataFrame, column: str, label: str) -> None:
    assert isinstance(df, pd.DataFrame) 
    assert isinstance(column, str)
    assert isinstance(label, str)
    genders = df.GENDER.tolist()
    data = {f"{label}":0, f"Not {label}":0}
    for v in df[column].tolist():
        if v == 1:
            data[f"{label}"] += 1
        else:
            data[f"Not {label}"] += 1
    labels = list(data.keys())
    values = list(data.values())
    width = .4
    colors = ["#104B8E", "#DC661A"]
    f, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))
    ax1.bar(labels, values, color=colors[0], width=width)
    ax1.set_xlabel("Outcome", size=16)
    ax1.set_ylabel("Number of patients", size=16)
    ax1.tick_params(labelsize=14)
    ax1.set_title(f"{label}", size=18)
    data = {f"{label}":[0,0], f"Not {label}":[0,0]}
    for i,v in enumerate(df[column].tolist()):
        if v == 1:
            if genders[i] == "M":
                data[f"{label}"][0] += 1
            else:  # genders[i] == "F"
                data[f"{label}"][1] += 1
        else:
            if genders[i] == "M":
                data[f"Not {label}"][0] += 1
            else:  # genders[i] == "F"
                data[f"Not {label}"][1] += 1
    x = np.arange(2)
    ax2.bar(x - width / 2, [data[f"{label}"][0], data[f"Not {label}"][0]], width, label="Males", color=colors[0])
    ax2.bar(x + width / 2, [data[f"{label}"][1], data[f"Not {label}"][1]], width, label="Females", color=colors[1])
    ax2.set_xlabel("Outcome", size=16)
    ax2.set_ylabel("Number of patients", size=16)
    ax2.tick_params(labelsize=14)
    ax2.set_xticks(x)
    ax2.set_xticklabels(list(data.keys()))
    ax2.legend(["Male", "Female"], prop={"size": 18})
    ax2.set_title(f"{label} by gender", size=18)
    plt.show()

### Load and explore data

#### ER Unit data

In [ ]:
ps_df = pd.read_csv(ps_data_path, sep=";")
# store column value description line
vars_description_ps = ps_df.iloc[1,:]
ps_df.head()

In [ ]:
# remove lines repeating the header and describing vars
ps_df.drop(ps_df.head(3).index, axis=0, inplace=True)
ps_df.reset_index(drop=True, inplace=True)
# drop columns with header "####" (separators)
drop_cols = [col for col in ps_df.columns.tolist() if "###" in col]
ps_df.drop(drop_cols, axis=1, inplace=True)
ps_df.head()

In [ ]:
# drop columns
keep_cols = [
    "CODE",
    "GENDER",
    "BIRTHDAY",
    "DEAD_DATE",
    "ID",
    "START",
    "STOP",
    "FIELDSET_PS-O2",
    "FIELDSET_PS-FR",
    "FIELDSET_PS-FC",
    "FIELDSET_PS-SYSTOLIC",
    "FIELDSET_PS-DIASTOLIC",
    "FIELDSET_PS-TEMPERATURE",
    "FIELDSET_PS-DYSPNEA",
    "FIELDSET_PS-FEVER",
    "FIELDSET_PS-COUGH",
    "FIELDSET_PS-GS",
    "FIELDSET_PS-ASTHENIA",
    "FIELDSET_PS-DATE",
    "FIELDSET_PS-ILLNESSES_APR",
    "FIELDSET_PS-ILLNESSES_1",
    "FIELDSET_PS-ILLNESSES_2",
    "FIELDSET_PS-ILLNESSES_3",
    "FIELDSET_PS-ILLNESSES_4",
    "FIELDSET_PS-ILLNESSES_5",
    "FIELDSET_PS-ILLNESSES_6",
    "FIELDSET_PS-ILLNESSES_7",
    "FIELDSET_PS-ILLNESSES_8",
    "FIELDSET_PS-ILLNESSES_9",
    "FIELDSET_PS-ILLNESSES_10",
    "FIELDSET_PS-ILLNESSES_11",
    "FIELDSET_PS-ILLNESSES_12",
    "FIELDSET_PS-ILLNESSES_13",
    "FIELDSET_PS-BLOOD_EGA_PH",
    "FIELDSET_PS-BLOOD_EGA_PO2",
    "FIELDSET_PS-BLOOD_EGA_PCO2",
    "FIELDSET_PS-BLOOD_EGA_FI02",
    "FIELDSET_PS-BLOOD_EGA_PF",
    "FIELDSET_PS-BLOOD_EGA_LATTATI",
    "FIELDSET_PS-BLOOD_COUNT_HB",
    "FIELDSET_PS-BLOOD_COUNT_HT",
    "FIELDSET_PS-BLOOD_COUNT_MCV",
    "FIELDSET_PS-BLOOD_COUNT_RDW",
    "FIELDSET_PS-BLOOD_COUNT_PLT",
    "FIELDSET_PS-BLOOD_COUNT_LEUCOCITI",
    "FIELDSET_PS-BLOOD_COUNT_NEUTROFILI",
    "FIELDSET_PS-BLOOD_COUNT_LINFOCITI",
    "FIELDSET_PS-BLOOD_PCR",
    "FIELDSET_PS-BLOOD_CREATININA",
    "FIELDSET_PS-BLOOD_EGFR",
    "FIELDSET_PS-BLOOD_NA",
    "FIELDSET_PS-BLOOD_K",
    "FIELDSET_PS-BLOOD_AST",
    "FIELDSET_PS-BLOOD_ALT",
    "FIELDSET_PS-BLOOD_LDH",
    "FIELDSET_PS-BLOOD_CK",
    "FIELDSET_PS-BLOOD_ALBUMINA",
    "FIELDSET_PS-BLOOD_BILIRUBINA",
    "FIELDSET_PS-BLOOD_FIBRINOGENO",
    "FIELDSET_PS-BLOOD_FERRITINA",
    "FIELDSET_PS-BLOOD_LIPASI",
    "FIELDSET_PS-BLOOD_PROTEINE",
    "FIELDSET_PS-BLOOD_UREA",
    "FIELDSET_PS-BLOOD_DDIMERO",
    "FIELDSET_PS-BLOOD_PT",
    "FIELDSET_PS-BLOOD_APTT",
    "FIELDSET_PS-SCORE_GCS",
    "FIELDSET_PS-DIMISSION",
    "FIELDSET_PHYSIOLOGICAL-HOME",
    "FIELDSET_PHYSIOLOGICAL-SMOKE",
    "FIELDSET_PHYSIOLOGICAL-ALCOHOL",
    "FIELDSET_CLINICAL-BARTHEL",
    "FIELDSET_VENTILATION-EXECUTED",
    "FIELDSET_EVOLUTION-LEGIONELLA",
    "FIELDSET_EVOLUTION-PNEUMOCOCCO",
    "FIELDSET_EVOLUTION-ARDS",
    "FIELDSET_EVOLUTION-PNEUMONIA",
    "FIELDSET_EVOLUTION-CARDIAC",
    "FIELDSET_EVOLUTION-THROMBOTIC",
    "FIELDSET_EVOLUTION-HEMORRAGICALS",
    "FIELDSET_EVOLUTION-TRANSFUSION_BLOOD",
    "FIELDSET_EVOLUTION-TRANSFUSION_PLASMA",
    "FIELDSET_EVOLUTION-TRANSFUSION_PLATES",
    "FIELDSET_EVOLUTION-RENALS",
    "FIELDSET_EVOLUTION-HEPATICS",
    "FIELDSET_EVOLUTION-INFECTIOUS",
    "FIELDSET_EVOLUTION-NEUROLOGICAL",
    "FIELDSET_EVOLUTION-DERMATOLOGY",
    "FIELDSET_EVOLUTION-O2",
]
ps_df_filter = ps_df[keep_cols]
ps_df_filter.head()

In [ ]:
# check NA values pct on columns
check_cols = [
    "FIELDSET_PS-FR",
    "FIELDSET_PS-FC",
    "FIELDSET_PS-SYSTOLIC",
    "FIELDSET_PS-DIASTOLIC",
    "FIELDSET_PS-TEMPERATURE",
    "FIELDSET_PS-BLOOD_DDIMERO",
    "FIELDSET_PS-SCORE_GCS",
    "FIELDSET_EVOLUTION-LEGIONELLA",
    "FIELDSET_EVOLUTION-PNEUMOCOCCO"
]
drop_cols = []
for col in check_cols:
    nan_pct = compute_nan_pct(ps_df_filter[col].tolist())
    print(f"NaN percentage in column {col}: {nan_pct}")
    if nan_pct > 0.5:
        drop_cols.append(col)
# FIELDSET_PS-FR: 0.56
# FIELDSET_PS-BLOOD_DDIMERO: 0.95
ps_df_filter.drop(drop_cols, axis=1, inplace=True)
ps_df_filter.head()

In [ ]:
# set NaN values to 0
check_cols = [
    "FIELDSET_PS-DYSPNEA",
    "FIELDSET_PS-FEVER",
    "FIELDSET_PS-COUGH",
    "FIELDSET_PS-GS",
    "FIELDSET_PS-ASTHENIA",
    "FIELDSET_PS-ILLNESSES_1",
    "FIELDSET_PS-ILLNESSES_2",
    "FIELDSET_PS-ILLNESSES_3",
    "FIELDSET_PS-ILLNESSES_4",
    "FIELDSET_PS-ILLNESSES_5",
    "FIELDSET_PS-ILLNESSES_6",
    "FIELDSET_PS-ILLNESSES_7",
    "FIELDSET_PS-ILLNESSES_8",
    "FIELDSET_PS-ILLNESSES_9",
    "FIELDSET_PS-ILLNESSES_10",
    "FIELDSET_PS-ILLNESSES_11",
    "FIELDSET_PS-ILLNESSES_12",
    "FIELDSET_PS-ILLNESSES_13",
    "FIELDSET_EVOLUTION-ARDS",
    "FIELDSET_EVOLUTION-PNEUMONIA",
    "FIELDSET_EVOLUTION-CARDIAC",
    "FIELDSET_EVOLUTION-THROMBOTIC",
    "FIELDSET_EVOLUTION-HEMORRAGICALS",
    "FIELDSET_EVOLUTION-TRANSFUSION_BLOOD",
    "FIELDSET_EVOLUTION-TRANSFUSION_PLASMA",
    "FIELDSET_EVOLUTION-TRANSFUSION_PLATES",
    "FIELDSET_EVOLUTION-RENALS",
    "FIELDSET_EVOLUTION-HEPATICS",
    "FIELDSET_EVOLUTION-INFECTIOUS",
    "FIELDSET_EVOLUTION-NEUROLOGICAL",
    "FIELDSET_EVOLUTION-DERMATOLOGY"
]
df_cols = np.array(ps_df_filter.columns.tolist())
for col in check_cols:
    idx = int(np.where(df_cols == col)[0])
    ps_df_filter[col] = ps_df_filter.apply(lambda x : nan_to_zero(x[idx]), axis=1)
ps_df_filter.head()

In [ ]:
# remove columns with NaN values on START or STOP columns
starts = ps_df_filter.START.tolist()
stops = ps_df_filter.STOP.tolist()
assert len(starts) == len(stops)
drop_idxs = [i for i in range(len(starts)) if str(starts[i]) == "nan" or str(stops[i]) == "nan"]
ps_df_filter.drop(drop_idxs, axis=0, inplace=True)

# compute disease duration
ps_df_filter["COVID_infection_duration"] = ps_df_filter.apply(lambda x : compute_date_diff(x[5], x[6]), axis=1)
ps_df_filter.head()

In [ ]:
# define the response column (DEAD_DATE)
ps_df_filter["DEATH"] = ps_df_filter.apply(lambda x : compute_death_response(str(x[3])), axis=1)
ps_df_filter.head()

In [ ]:
ps_df_filter.shape  # 787 patients (89 variables)
ps_df_filter.describe()  # ID is the only variable with 788 unique values 

#### RX data

In [ ]:
rx_df = pd.read_csv(rx_data_path, sep=";")
# store column value description
vars_description_rx = rx_df.iloc[1,:]
rx_df.head()

In [ ]:
# remove lines repeating header names
rx_df.drop(rx_df.head(3).index, axis=0, inplace=True)
# remove columns named "####" (separators)
drop_cols = [col for col in rx_df.columns.tolist() if "###" in col]
rx_df.drop(drop_cols, axis=1, inplace=True)
rx_df.head()

In [ ]:
# drop columns
keep_cols = [
    "CODE", "ID", "START", "FIELDSET_RX-0-RX_1", "FIELDSET_RX-0-RX_2", "FIELDSET_RX-0-RX_3", "FIELDSET_RX-0-RX_4"
]
rx_df_filter = rx_df[keep_cols]
rx_df_filter.head()

In [ ]:
rx_df_filter.shape  # 942 patients (7 variables)
rx_df.describe()  # again ID is the only variable with 942 unique values

#### Merge ER and RX datasets

In [ ]:
# check how many ER dataset's visits are in the RX dataset
visit_ps = set(ps_df_filter.ID.tolist())
visit_rx = set(rx_df_filter.ID.tolist())
len(visit_ps.intersection(visit_rx))  # 769 --> the additional 18 visits should be removed

In [ ]:
ps_df_filter = ps_df_filter[ps_df_filter.ID.isin(visit_ps.intersection(visit_rx))]
rx_df_filter = rx_df_filter[rx_df_filter.ID.isin(visit_ps.intersection(visit_rx))]
ps_rx_df = ps_df_filter.merge(rx_df_filter, on="ID")
ps_rx_df.head()

In [ ]:
ps_rx_df.shape  # merged dataset --> 769 ER accesses and 95 total variables
ps_rx_df.describe()

In [ ]:
# search and remove repeated columns
drop_cols = [col for col in ps_rx_df.columns.tolist() if "_x" in col or "_y" in col]
keep_cols = [col for col in drop_cols if "_x" in col]
for col in keep_cols:
    ps_rx_df[col.split("_")[0]] = ps_rx_df[col]
ps_rx_df.drop(drop_cols, axis=1, inplace=True)
ps_rx_df.shape  # number of variables decreased to 93

In [ ]:
# display the final dataset
ps_rx_df.head()

#### Plotting some stats on our data

In [ ]:
# explore the outcomes
data = {
    "Dead":ps_rx_df[ps_rx_df.DEATH == 1].shape[0], 
    "Alive":ps_rx_df[ps_rx_df.DEATH == 0].shape[0]
}
outcome = list(data.keys())
values = list(data.values())
f,(ax1,ax2) = plt.subplots(1,2,figsize = (20, 10))
ax1.bar(outcome, values, color = "#104B8E", width = 0.4)
ax1.set_xlabel("Outcome", size=16)
ax1.set_ylabel("Number of patients", size=16)
ax1.tick_params(labelsize=14)
ax1.set_title("ER visits outcome", size=18)
colors = ["#104B8E", "#DC661A"]
data = {
    "Dead":[
        ps_rx_df[ (ps_rx_df.DEATH == 1) & (ps_rx_df.GENDER == "M") ].shape[0],
        ps_rx_df[ (ps_rx_df.DEATH == 1) & (ps_rx_df.GENDER == "F") ].shape[0]
    ], 
    "Alive":[
        ps_rx_df[ (ps_rx_df.DEATH == 0) & (ps_rx_df.GENDER == "M") ].shape[0],
        ps_rx_df[ (ps_rx_df.DEATH == 0) & (ps_rx_df.GENDER == "F") ].shape[0]
    ]
}
x = np.arange(2)
width = 0.2
ax2.bar(x - width / 2, [data["Dead"][0], data["Alive"][0]], width, label="Males", color=colors[0])
ax2.bar(x + width / 2, [data["Dead"][1], data["Alive"][1]], width, label="Females", color=colors[1])
ax2.set_xlabel("Outcome", size=16)
ax2.set_ylabel("Number of patients", size=16)
ax2.tick_params(labelsize=14)
ax2.set_xticks(x)
ax2.set_xticklabels(list(data.keys()))
ax2.legend(["Male", "Female"], prop={"size": 18})
ax2.set_title("ER visits outcome by gender", size=18)
plt.show()

In [ ]:
data = {
    "< 5 days":0,
    "5 - 15 days":0,
    "16 - 30 days":0,
    "> 30 days":0
}
for duration in ps_rx_df.COVID_infection_duration.tolist():
    if int(duration) < 5:
        data["< 5 days"] += 1
    elif int(duration) >= 5 and int(duration) < 16:
        data["5 - 15 days"] += 1
    elif int(duration) >= 16 and int(duration) < 31:
        data["16 - 30 days"] += 1
    else:  # > 30 days
        data["> 30 days"] += 1
duration = list(data.keys())
values = list(data.values())
f, (ax1, ax2) = plt.subplots(1,2,figsize=(20,10))
ax1.bar(duration, values, color="#104B8E", width=.4)
ax1.set_xlabel("COVID duration", size=16)
ax1.set_ylabel("Number of patients", size=16)
ax1.tick_params(labelsize=14)
ax1.set_title("COVID infection duration", size=18)
colors = ["#104B8E", "#DC661A"]
data = {
    "< 5 days":[0,0],
    "5 - 15 days":[0,0],
    "16 - 30 days":[0,0],
    "> 30 days":[0,0]
}
genders = ps_rx_df.GENDER.tolist()
for i,duration in enumerate(ps_rx_df.COVID_infection_duration.tolist()):
    if int(duration) < 5:
        if genders[i] == "M":
            data["< 5 days"][0] += 1
        else:  # genders[i] == "F"
            data["< 5 days"][1] += 1
    elif int(duration) >= 5 and int(duration) < 16:
        if genders[i] == "M":
            data["5 - 15 days"][0] += 1
        else:  # genders[i] == "F"
            data["5 - 15 days"][1] += 1
    elif int(duration) >= 16 and int(duration) < 31:
        if genders[i] == "M":
            data["16 - 30 days"][0] += 1
        else:  # genders[i] == "F"
            data["16 - 30 days"][1] += 1
    else:  # > 30 days
        if genders[i] == "M":
            data["> 30 days"][0] += 1
        else:  # genders[i] == "F"
            data["> 30 days"][1] += 1
x = np.arange(len(data.keys()))
width = 0.3
males_data = [data[d][0] for d in data.keys()]
females_data = [data[d][1] for d in data.keys()]
ax2.bar(x - width / 2, males_data, width, label="Males", color=colors[0])
ax2.bar(x + width / 2, females_data, width, label="Females", color=colors[1])
ax2.set_xlabel("COVID duration", size=16)
ax2.set_ylabel("Number of patients", size=16)
ax2.tick_params(labelsize=14)
ax2.set_xticks(x)
ax2.set_xticklabels(list(data.keys()))
ax2.legend(["Male", "Female"], prop={"size": 18})
ax2.set_title("COVID infection duration by gender", size=18)
plt.show()

### Store the processed and merged data

In [ ]:
ps_rx_df.to_csv(
    os.path.join(data_path, "merged_data_processed.csv"), 
    index=False
)